<a href="https://colab.research.google.com/github/acse-srm3018/DeeplearningProxy/blob/main/Saturation_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Fri Jul  9 13:08:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [23]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [25]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [26]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [27]:
downloaded = drive.CreateFile({'id':"1A_mWXlDPWwfkjpD6I3faBn3yz_z2KHOh"})   # replace the id with id of file you want to access
#downloaded.GetContentFile('') 


In [6]:
import numpy as np
def load_data(path):
    """
    Function to load datasets in format .NPY
    
    Parameter:
    ----------
    path : string
        The absolute path of where data saved in local system
        
    Return:
    ----------
    loaded_data : ndarray
        The data which was loaded
    """
    loaded_data = np.load(path)
    return loaded_data

In [8]:
# define the absolute path of training datatsat
path_x = '/content/gdrive/MyDrive/Abder_Intersect_Simulations_5_spots/All_X.npy'
path_target = '/content/gdrive/MyDrive/Abder_Intersect_Simulations_5_spots/All_Y.npy'
max_path = '/content/gdrive/MyDrive/Abder_Intersect_Simulations_5_spots/Max_Pressure.npy'
# use load_data function nd above path to loading data
X_data = load_data(path_x)
target_data = load_data(path_target)
Max_pressure = load_data(max_path)

In [9]:
# Split datasets to porosity, permeability and pressurre and ssaturation 
permeability = X_data[:, : , : , :, 0]
porosity = X_data[:, : , : , :, 1]
saturation = target_data[:, :, :, :, 1]
pressure = target_data[:, :, :, :, 0]

In [11]:
!git clone https://github.com/acse-srm3018/DeeplearningProxy.git

Cloning into 'DeeplearningProxy'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 90 (delta 33), reused 3 (delta 0), pack-reused 0
Unpacking objects: 100% (90/90), done.


In [37]:
from google.colab import files
files.upload()
import sys
sys.path.append('/content/gdrive/MyDrive/Colab Notebooks/layers.py')
sys.path.append('/content/gdrive/MyDrive/Colab Notebooks/unet_uae.py')

Saving unet_uae.py to unet_uae.py


In [38]:
import numpy as np
import unet_uae as vae_util
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from tensorflow.keras import layers
from tensorflow.keras.models import load_model, Model
from tensorflow.python.keras import backend as K
from keras.optimizers import Adam
import tensorflow as tf 
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.3
sess = tf.compat.v1.Session(config=config)
K.set_session(sess)

In [39]:
input_shape=(100, 100, 2)
depth = 7
vae_model,_ = vae_util.create_vae(input_shape, depth)
vae_model.summary()

output shape is  (None, 7, 100, 100, 1)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image (InputLayer)              [(None, 100, 100, 2) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 50, 50, 16)   304         image[0][0]                      
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 50, 50, 16)   64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 50, 50, 16)   0           batch_normalization[0][0]        
______________________________________________________

In [40]:
step_index = [1, 2, 4, 6, 8, 10, 12]
depth = 7
nr = permeability.shape[0]
train_nr = 2250
test_nr = 750
train_x = np.concatenate([permeability[:train_nr,[0], ...],saturation[:train_nr,[0], ...]], axis = 1)
train_y = saturation[:train_nr,step_index,...]

test_x = np.concatenate([permeability[nr-test_nr:,[0], ...], saturation[nr-test_nr:, [0], ...]], axis = 1)
test_y = saturation[nr-test_nr:,step_index,...]


train_x = train_x.transpose(0,2,3,1)
train_y = train_y[:,:,:,:,None]
test_x = test_x.transpose(0,2,3,1)
test_y = test_y[:,:,:,:,None]
#test_y = test_y.transpose(0,2,3,1)
print('train_x shape is ', train_x.shape)
print('train_y shape is ', train_y.shape)
print('test_x shape is ', test_x.shape)
print('test_y shape is ', test_y.shape)


train_x shape is  (2250, 100, 100, 2)
train_y shape is  (2250, 7, 100, 100, 1)
test_x shape is  (750, 100, 100, 2)
test_y shape is  (750, 7, 100, 100, 1)
